# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:
이번 연습문제는 SVM 연습문제와 유사함

- implement a fully-vectorized **loss function** for the Softmax classifier  
  (소프트맥스 분류기를 위한 완전한 병렬 ** 손실함수 ** 를 작성하라)
- implement the fully-vectorized expression for its **analytic gradient**  
  ( **analystic 미분** 를 위한 full-vectorize 로 구현하라)
- **check your implementation** with numerical gradient  
  (** 구현 결과물을 검증**을 수치적 미분으로 수행하라)
- use a validation set to **tune the learning rate and regularization** strength  
  (** 학습률과 정규화 세부조정** 을 위해 검증셋을 이용하라) 
- **optimize** the loss function with **SGD**  
  (** SGD ** 를 이용해 손실 함수를 ** 최적화** 하라)
- **visualize** the final learned weights  
  (최종 학습 계수를 ** 시각화 ** 하라)


In [2]:
%%javascript
var fname = document.body.attributes['data-notebook-name'].value
var fpath = document.body.attributes['data-notebook-path'].value.replace(/^local/, '/root').replace(fname, '')
IPython.notebook.kernel.execute("theNoteBookDir = '" + fpath + "'" );

<IPython.core.display.Javascript object>

In [3]:
import os
os.chdir(theNoteBookDir)

In [4]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [5]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
  """
  Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
  it for the linear classifier. These are the same steps as we used for the
  SVM, but condensed to a single function.  
  """
  # Load the raw CIFAR-10 data
  cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
  X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
  
  # subsample the data
  mask = range(num_training, num_training + num_validation)
  X_val = X_train[mask]
  y_val = y_train[mask]
  mask = range(num_training)
  X_train = X_train[mask]
  y_train = y_train[mask]
  mask = range(num_test)
  X_test = X_test[mask]
  y_test = y_test[mask]
  mask = np.random.choice(num_training, num_dev, replace=False)
  X_dev = X_train[mask]
  y_dev = y_train[mask]
  
  # Preprocessing: reshape the image data into rows
  X_train = np.reshape(X_train, (X_train.shape[0], -1))
  X_val = np.reshape(X_val, (X_val.shape[0], -1))
  X_test = np.reshape(X_test, (X_test.shape[0], -1))
  X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
  
  # Normalize the data: subtract the mean image
  mean_image = np.mean(X_train, axis = 0)
  X_train -= mean_image
  X_val -= mean_image
  X_test -= mean_image
  X_dev -= mean_image
  
  # add bias dimension and transform into columns
  X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
  X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
  X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
  X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
  
  return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print 'Train data shape: ', X_train.shape
print 'Train labels shape: ', y_train.shape
print 'Validation data shape: ', X_val.shape
print 'Validation labels shape: ', y_val.shape
print 'Test data shape: ', X_test.shape
print 'Test labels shape: ', y_test.shape
print 'dev data shape: ', X_dev.shape
print 'dev labels shape: ', y_dev.shape

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside **cs231n/classifiers/softmax.py**. 


In [6]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print 'loss: %f' % loss
print 'sanity check: %f' % (-np.log(0.1))

loss: 2.375229
sanity check: 2.302585


## Inline Question 1:
Why do we expect our loss to be close to -log(0.1)? Explain briefly.
loss 가 문제 없는지 알기 위해 왜  -log(0.1) 과 비교 했을까?

**Your answer:** *Fill this in*
클래스가 10개 이고 랜덤하게 W를 주었으므로 ground truth 클래스의 score 값이 전체의 0.1 이었을 것이라 추정하는 것이 자연스럽다.


In [7]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

print '\ntry again. another gradient check with regularization\n'

# similar to SVM case, do another gradient check with regularization
# SVM에서 그랬던 것 처럼, 정규화 성분을 넣고 미분 검증을 수행해라
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 1e2)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 1e2)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 1.441099 analytic: 1.441099, relative error: 1.712287e-08
numerical: 0.183150 analytic: 0.183150, relative error: 4.583481e-08
numerical: 0.263879 analytic: 0.263879, relative error: 1.726340e-07
numerical: -3.021855 analytic: -3.021855, relative error: 3.178625e-09
numerical: -2.267320 analytic: -2.267320, relative error: 6.008682e-10
numerical: -0.561620 analytic: -0.561620, relative error: 5.235268e-08
numerical: 1.752655 analytic: 1.752655, relative error: 2.167065e-08
numerical: 0.768413 analytic: 0.768413, relative error: 3.656334e-08
numerical: -0.244252 analytic: -0.244252, relative error: 2.628522e-07
numerical: 0.456599 analytic: 0.456599, relative error: 8.257525e-08

try again. another gradient check with regularization

numerical: 0.761263 analytic: 0.761262, relative error: 8.417498e-08
numerical: 0.728728 analytic: 0.728728, relative error: 3.549163e-09
numerical: 3.109773 analytic: 3.109772, relative error: 3.425621e-08
numerical: -3.396360 analytic: -3.39636

In [8]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
# 백터 버전을 만들어보아라
# 동일한 결과를 내어아야 하고 더 많이 빨라야 한다.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.00001)
toc = time.time()
print 'naive loss: %e computed in %fs' % (loss_naive, toc - tic)

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
#loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.00001)
loss_vectorized, grad_vectorized = softmax_loss_naive(W, X_dev, y_dev, 0.00001)

toc = time.time()
print 'vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic)

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
# 두 미분 버전을 비교하기 위해 Frobenius norm을 사용한다
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print 'Loss difference: %f' % np.abs(loss_naive - loss_vectorized)
print 'Gradient difference: %f' % grad_difference

naive loss: 2.375229e+00 computed in 0.134647s
vectorized loss: 2.375229e+00 computed in 0.118971s
Loss difference: 0.000000
Gradient difference: 0.000000


In [10]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 5e-7]
regularization_strengths = [5e4, 1e8]

################################################################################
# TODO: 해야 할일                                                              #
# Use the validation set to set the learning rate and regularization strength. #
# 학습률과 정규화 강도를 설정하기 위해 검증셋을 사용해라                       #
# This should be identical to the validation that you did for the SVM; save    #
# 이 검증셋은 SVM 에서 사용한 검증셋과 동일해야 한다.                          #
# the best trained softmax classifer in best_softmax.                          #
# 가장 좋은 분류기를 저장하라                                                  #
################################################################################

# learning rate과 regularization_strength 를 입력 받아
# 학습후에 train 정확도와 validation 정확도를 출력한다.
def calcu_accuracy(learning_rate, regularization_strength, svm) :    
    svm.train(X_train, y_train, learning_rate, regularization_strength, num_iters=2000)
    y_train_pred = svm.predict(X_train)
    y_val_pred   = svm.predict(X_val)
    training_accuracy   = np.mean(y_train == y_train_pred)
    validation_accuracy = np.mean(y_val   == y_val_pred)
    return training_accuracy, validation_accuracy

def sample(rng1, rng2, cnt):
    sample_rng1 = np.random.uniform(rng1[0], rng1[1], cnt)
    sample_rng2 = np.random.uniform(rng2[0], rng2[1], cnt)    
    return zip(sample_rng1, sample_rng2)

def to_exp(p1, p2):
    return 10**p1, 10**p2


def search(lr_rng, rs_rng):
    global best_val
    print '------------------------------------------------------------'
    print 'learning rate range:[%.5f, %.5f]'%(lr_rng[0], lr_rng[1]), ', regularizatin range:[%.5f, %.5f]'%(rs_rng[0], rs_rng[1])
    tic = time.time()
    
    hypers = sample(lr_rng, rs_rng, 50)
    results_temp = {}    
    for hyper_param in hypers: 
        svm = Softmax()
        exp_param = to_exp(*hyper_param)
        results_temp[hyper_param] = calcu_accuracy(exp_param[0], exp_param[1], svm)
        sys.stdout.write('>')
        cur_val = results_temp[hyper_param][1]
        if best_val < cur_val:
            best_val  = cur_val
            best_svm  = svm
            sys.stdout.write('<#')
    sys.stdout.write('\n')
    results.update(results_temp)
    max_key = max(results.iterkeys(),      key=(lambda key: results[key][1]))
    min_key = min(results_temp.iterkeys(), key=(lambda key: results_temp[key][1]))
    
    lr_rng = [(2*min_key[0] + max_key[0])/3, (3*max_key[0] - min_key[0])/2]
    rs_rng = [(2*min_key[1] + max_key[1])/3, (3*max_key[1] - min_key[1])/2]
    
    
    print '[min] ', 'lr:%.5f, rs:%.5f'% min_key, ', train_acc:%.5f, val_acc:%.5f'%results_temp[min_key]
    print '[max] ', 'lr:%.5f, rs:%.5f'% max_key, ', train_acc:%.5f, val_acc:%.5f'%results[max_key]
    toc = time.time()
    print '[time] %.0fs' % (toc - tic)
    return lr_rng, rs_rng

# learning_rates = [1e-7, 5e-5]
# regularization_strengths = origianl [5e4, 1e5]
lr_rng = [-8, -3]
rs_rng = [-5,  5]
for _ in xrange(4):
    lr_rng, rs_rng = search(lr_rng, rs_rng)
    
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print 'lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy)
    
print 'best validation accuracy achieved during cross-validation: %f' % best_val

------------------------------------------------------------
learning rate range:[-8.00000, -3.00000] , regularizatin range:[-5.00000, 5.00000]


NameError: global name 'W1' is not defined

In [32]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print 'softmax on raw pixels final test set accuracy: %f' % (test_accuracy, )

AttributeError: 'NoneType' object has no attribute 'predict'

In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in xrange(10):
  plt.subplot(2, 5, i + 1)
  
  # Rescale the weights to be between 0 and 255
  wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
  plt.imshow(wimg.astype('uint8'))
  plt.axis('off')
  plt.title(classes[i])